<a href="https://colab.research.google.com/github/aderdouri/PINNs/blob/master/Tutorials/IPINN_Navier_Stokes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

# Helper function for gradient computation
def gradients(y, x):
    return torch.autograd.grad(y, x, grad_outputs=torch.ones_like(y), create_graph=True, retain_graph=True)[0]

# Define the PINN model
class PINN(nn.Module):
    def __init__(self):
        super(PINN, self).__init__()
        self.hidden_layers = nn.Sequential(
            nn.Linear(3, 50),  # Input: (t, x, y)
            nn.Tanh(),
            nn.Linear(50, 50),
            nn.Tanh(),
            nn.Linear(50, 50),
            nn.Tanh(),
            nn.Linear(50, 3)   # Output: (v_x, v_y, p)
        )

    def forward(self, t, x, y):
        inputs = torch.cat([t, x, y], dim=1)
        return self.hidden_layers(inputs)

# Define the unknown parameters (C1 and C2)
log_C1 = torch.tensor([0.0], requires_grad=True, dtype=torch.float32)  # log(C1) for positivity
log_C2 = torch.tensor([0.0], requires_grad=True, dtype=torch.float32)  # log(C2) for positivity

# Initialize the model
model = PINN()

# Loss function
def loss_function(t, x, y, v_obs, p_obs):
    C1 = torch.exp(log_C1)
    C2 = torch.exp(log_C2)

    # Ensure gradients can be computed
    t.requires_grad_(True)
    x.requires_grad_(True)
    y.requires_grad_(True)

    # Forward pass
    output = model(t, x, y)
    v_x, v_y, p = output[:, 0:1], output[:, 1:2], output[:, 2:3]

    # Compute derivatives
    v_x_t = gradients(v_x, t)
    v_x_x = gradients(v_x, x)
    v_x_y = gradients(v_x, y)
    v_xx = gradients(v_x_x, x)
    v_yy = gradients(v_x_y, y)

    v_y_t = gradients(v_y, t)
    v_y_x = gradients(v_y, x)
    v_y_y = gradients(v_y, y)
    v_yx = gradients(v_y_x, x)
    v_yy = gradients(v_y_y, y)

    p_x = gradients(p, x)
    p_y = gradients(p, y)

    # Residuals of Navier-Stokes equations
    f_vx = v_x_t + C1 * (v_x * v_x_x + v_y * v_x_y) + p_x - C2 * (v_xx + v_yy)
    f_vy = v_y_t + C1 * (v_x * v_y_x + v_y * v_y_y) + p_y - C2 * (v_yx + v_yy)

    # Physics-based loss
    physics_loss = torch.mean(f_vx**2) + torch.mean(f_vy**2)

    # Data loss (mean squared error with observed data)
    data_loss = torch.mean((v_x - v_obs[:, 0:1])**2) + \
                torch.mean((v_y - v_obs[:, 1:2])**2) + \
                torch.mean((p - p_obs)**2)

    # Total loss
    total_loss = physics_loss + data_loss
    return total_loss

# Optimizer for both model parameters and unknown constants
optimizer = optim.Adam(list(model.parameters()) + [log_C1, log_C2], lr=0.001)

# Training data (replace with actual data)
n_points = 1000
x_train = torch.rand(n_points, 1, requires_grad=True)
y_train = torch.rand(n_points, 1, requires_grad=True)
t_train = torch.rand(n_points, 1, requires_grad=True)
v_train = torch.cat([torch.sin(torch.pi * x_train), torch.cos(torch.pi * y_train)], dim=1)  # Example: true velocities
p_train = torch.zeros_like(x_train)  # Example: true pressure

# Training loop
n_epochs = 2000
for epoch in range(n_epochs):
    optimizer.zero_grad()
    loss = loss_function(t_train, x_train, y_train, v_train, p_train)
    loss.backward()
    optimizer.step()

    if epoch % 100 == 0:
        print(f"Epoch {epoch}, Loss: {loss.item():.6f}, C1: {torch.exp(log_C1).item():.6f}, C2: {torch.exp(log_C2).item():.6f}")

print("Training complete!")
print(f"Learned C1: {torch.exp(log_C1).item():.6f}")
print(f"Learned C2: {torch.exp(log_C2).item():.6f}")
